# Adversarial Attacks and Robustness ｜ FGSM

This demo demonstrates how to use the **fast gradient sign method  (FSGM)** [1] to generate adversarial examples. 

<img src="Util/demo2_1.jpg" alt="white-box attack" width="500"/>

[1] Goodfellow, Ian J., Jonathon Shlens, and Christian Szegedy. "Explaining and harnessing adversarial examples." ICLR 2015.

### Step 1: Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import matplotlib.pyplot as plt
import numpy as np

### Step 2: Prepare the MNIST Dataset

In [2]:
# Transform for MNIST: Resize to 224x224 to match ResNet18 input
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

# Load MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='Data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='Data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

AttributeError: module 'torch' has no attribute '_six'

### Step 3: Define the Model